# Pré processamento da coluna SUMMARY

In [118]:
import pandas as pd
import unicodedata
import string

In [119]:
try:
    df = pd.read_csv(
        '../data/tasks.csv',
        encoding='utf-8',
        sep=",",
        quotechar='"',
        on_bad_lines='skip',      # ignora linhas corrompidas
        dtype=str,               # carrega tudo como string, depois ajusta
        engine='c'          # usa engine python que é mais tolerante
    )
    print(f"CSV carregado com engine C! {len(df)} linhas encontradas.")

except Exception as e:
    print(f"Erro ao ler CSV: {e}")

CSV carregado com engine C! 7232 linhas encontradas.


In [120]:
# Verificar informações básicas do dataset
print("=== INFORMAÇÕES BÁSICAS DO DATASET ===")
print(f"Shape: {df.shape}")
# Informações apenas da coluna SUMMARY
print(f"\nResumo da coluna SUMMARY:")
print(f"  - Tipo de dado: {df['SUMMARY'].dtype}")
print(f"  - Valores únicos: {df['SUMMARY'].nunique()}")
print(f"  - Valores nulos: {df['SUMMARY'].isnull().sum()}")
print(f"  - Primeiros 5 valores:\n{df['SUMMARY'].head()}")

=== INFORMAÇÕES BÁSICAS DO DATASET ===
Shape: (7232, 17)

Resumo da coluna SUMMARY:
  - Tipo de dado: object
  - Valores únicos: 376
  - Valores nulos: 0
  - Primeiros 5 valores:
0     🍳
1     🧋
2    🍽️
3     🥪
4     🍌
Name: SUMMARY, dtype: object


### Análise

In [121]:
print("=== ANÁlISE DA COLUNA SUMMARY ===")

print(f"Quantidade de valores únicos: {df['SUMMARY'].nunique()}")
# print(f"Valores únicos: {df['SUMMARY'].unique()}")

# Exportar valores únicos da coluna SUMMARY para uma tabela CSV e usar uma LLM para clusterizar informacoes parecidas
unique_summary = df[['SUMMARY', 'CALENDAR']].drop_duplicates().rename(columns={'SUMMARY': 'TITLE'})
unique_summary = unique_summary.sort_values(by='TITLE').reset_index(drop=True)
unique_summary.to_csv('../data/title_calendar_unique.csv', index=False, encoding='utf-8')
print("Valores únicos exportados para ../data/title_unique.csv")


=== ANÁlISE DA COLUNA SUMMARY ===
Quantidade de valores únicos: 376
Valores únicos exportados para ../data/title_unique.csv


## Limpando

In [122]:
# Avaliação antes da normalização
print("=== Avaliação da coluna SUMMARY antes da normalização ===")
print(f"Valores únicos: {df['SUMMARY'].nunique()}")
print(f"Exemplos: {df['SUMMARY'].drop_duplicates().head(10).tolist()}")

def normalize_text(text):
    if pd.isnull(text):
        return ""
    # Lowercase
    text = text.lower()
    # Remove accents
    # text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove extra spaces
    text = ' '.join(text.split())
    return text

df['SUMMARY_NORM'] = df['SUMMARY'].apply(normalize_text)

# Avaliação depois da normalização
print("\n=== Avaliação da coluna SUMMARY_NORM depois da normalização ===")
print(f"Valores únicos: {df['SUMMARY_NORM'].nunique()}")
print(f"Exemplos: {df['SUMMARY_NORM'].drop_duplicates().head(10).tolist()}")

=== Avaliação da coluna SUMMARY antes da normalização ===
Valores únicos: 376
Exemplos: ['🍳', '🧋', '🍽️', '🥪', '🍌', 'INF1410 - Gerenc Proj Inf', 'INF1608 - Análise Numérica', 'ADM1019 - Intr. Finanças', 'INF1629 - P. Eng. de Software', 'Parabéns!']

=== Avaliação da coluna SUMMARY_NORM depois da normalização ===
Valores únicos: 372
Exemplos: ['🍳', '🧋', '🍽️', '🥪', '🍌', 'inf1410 gerenc proj inf', 'inf1608 análise numérica', 'adm1019 intr finanças', 'inf1629 p eng de software', 'parabéns']


### Tratando valores da coluna SUMMARY que possuem dois valores na coluna CALENDAR

In [123]:
# Coluna SUMMARY possui 30 valores que tem duas variações em 'CALENDAR'
print("=== TRATANDO Duplicatas da coluna SUMMARY ===")
df['CALENDAR'] = df['CALENDAR'].replace('PUC 👻', 'PUC')

# Corrigindo os valores duplicados de 'Academia'
df.loc[(df['SUMMARY'] == 'Academia') & (df['CALENDAR'] == 'Pessoal'), 'CALENDAR'] = 'Exercícios' # 2
print(f'Valores de CALENDAR para "Academia": {df.loc[df['SUMMARY'] == 'Academia', 'CALENDAR'].unique()}')

# Corrigindo os valores duplicados de 'Aldeia'
df.loc[(df['SUMMARY'] == 'Aldeia') & (df['CALENDAR'] == 'Pessoal'), 'CALENDAR'] = 'Social' # 2
print(f'Valores de CALENDAR para "Aldeia": {df.loc[df['SUMMARY'] == 'Aldeia', 'CALENDAR'].unique()}')

# Corrigindo os valores duplicados de 'Alessandra'
df.loc[(df['SUMMARY'] == 'Alessandra') & (df['CALENDAR'] == 'Compromisso'), 'CALENDAR'] = 'Saúde' # 2
print(f'Valores de CALENDAR para "Alessandra": {df.loc[df["SUMMARY"] == "Alessandra", "CALENDAR"].unique()}')

# Corrigindo os valores duplicados de 'Aula'
df.loc[(df['SUMMARY'] == 'Aula') & (df['CALENDAR'] == 'Pessoal'), 'CALENDAR'] = 'PUC' # 2
print(f'Valores de CALENDAR para "Aula": {df.loc[df['SUMMARY'] == 'Aula', 'CALENDAR'].unique()}')

# Corrigindo os valores duplicados de 'BD: T2 G2'
df.loc[(df['SUMMARY'] == 'BD: T2 G2') & (df['CALENDAR'] == 'PUC'), 'CALENDAR'] = 'Exames' # 2
print(f'Valores de CALENDAR para "BD: T2 G2": {df.loc[df['SUMMARY'] == 'BD: T2 G2', 'CALENDAR'].unique()}')

# Corrigindo os valores duplicados de 'BIRUSAMBA DE VERÃO'
df.loc[(df['SUMMARY'] == 'BIRUSAMBA DE VERÃO'), 'CALENDAR'] = 'Social' # 2
print(f'Valores de CALENDAR para "BIRUSAMBA DE VERÃO": {df.loc[df['SUMMARY'] == 'BIRUSAMBA DE VERÃO', 'CALENDAR'].unique()}')

# Corrigindo os valores duplicados de De-Para
df.loc[(df['SUMMARY'] == 'De-Para') & (df['CALENDAR'] == 'Exames'), 'CALENDAR'] = 'PUC' # 2
print(f'Valores de CALENDAR para "De-Para": {df.loc[df["SUMMARY"] == "De-Para", "CALENDAR"].unique()}')

# Corrigindo os valores duplicados de 'Muay Thai'
df.loc[(df['SUMMARY'] == 'Muay Thai') & (df['CALENDAR'] == 'Pessoal'), 'CALENDAR'] = 'Exercícios' # 2
print(f'Valores de CALENDAR para "Muay Thai": {df.loc[df["SUMMARY"] == "Muay Thai", "CALENDAR"].unique()}')

# Corrigindo valores duplicados de 'Psicóloga'
df.loc[(df['SUMMARY'] == 'Psicóloga') & (df['CALENDAR'] == 'Compromisso'), 'CALENDAR'] = 'Saúde' # 2
print(f'Valores de CALENDAR para "Psicóloga": {df.loc[df["SUMMARY"] == "Psicóloga", "CALENDAR"].unique()}')

# Corrigindo valores duplicados de 'SO: P1' # 2
df.loc[(df['SUMMARY'] == 'SO: P1') & (df['CALENDAR'] == 'PUC'), 'CALENDAR'] = 'Exames' # 2
print(f'Valores de CALENDAR para "SO: P1": {df.loc[df["SUMMARY"] == "SO: P1", "CALENDAR"].unique()}')

=== TRATANDO Duplicatas da coluna SUMMARY ===
Valores de CALENDAR para "Academia": ['Exercícios']
Valores de CALENDAR para "Aldeia": ['Social']
Valores de CALENDAR para "Alessandra": ['Saúde']
Valores de CALENDAR para "Aula": ['PUC']
Valores de CALENDAR para "BD: T2 G2": ['Exames']
Valores de CALENDAR para "BIRUSAMBA DE VERÃO": ['Social']
Valores de CALENDAR para "De-Para": ['PUC']
Valores de CALENDAR para "Muay Thai": ['Exercícios']
Valores de CALENDAR para "Psicóloga": ['Saúde']
Valores de CALENDAR para "SO: P1": ['Exames']


In [124]:
print("=== ENCONTRO DE DUPLICATAS DA COLUNA SUMMARY ===")
duplicates = df[df.duplicated(subset='SUMMARY', keep=False)]
# Seleciona duplicatas de SUMMARY que aparecem em mais de um CALENDAR
duplicates_sum_cal = (
    duplicates[['SUMMARY', 'CALENDAR']]
    .drop_duplicates()
    .groupby('SUMMARY')
    .filter(lambda x: x['CALENDAR'].nunique() > 1)
    .sort_values(by=['SUMMARY', 'CALENDAR'], ascending=[True, True])
)
print(f"Quantidade de duplicatas encontradas: {duplicates_sum_cal.shape[0]}")
print(f"Duplicatas encontradas:\n{duplicates_sum_cal}")

=== ENCONTRO DE DUPLICATAS DA COLUNA SUMMARY ===
Quantidade de duplicatas encontradas: 2
Duplicatas encontradas:
       SUMMARY CALENDAR
4566  Itaipava  Pessoal
5951  Itaipava   Social


## Verificando valores parecidos

### Passando o valor 'Alessandra' da coluna SUMMARY para 'Psicóloga'

In [125]:
print('Quantidade de valores únicos antes da unificação:')
print(f"Valores únicos antes da unificação: {df['SUMMARY'].nunique()}")

print('=== TROCANDO DE "Alessandra" para "Psiquiatra" ===')
df.loc[df['SUMMARY'] == 'Alessandra', 'SUMMARY'] = 'Psiquiatra'

print('Quantidade de valores únicos antes da unificação:')
print(f"Valores únicos antes da unificação: {df['SUMMARY'].nunique()}")


psi_values = df[df['SUMMARY'].str.lower().str.startswith('psi', na=False)]['SUMMARY'].unique()
print(f"Valores que começam com 'psi' antes da unificação: {psi_values}")

print('=== TROCANDO DE "Psicóloga" para "Psicólogo" ===')
df.loc[df['SUMMARY'] == 'Psicóloga', 'SUMMARY'] = 'Psicólogo'

psi_values = df[df['SUMMARY'].str.lower().str.startswith('psi', na=False)]['SUMMARY'].unique()
print(f"Valores que começam com 'psi' depois da unificação: {psi_values}")


Quantidade de valores únicos antes da unificação:
Valores únicos antes da unificação: 376
=== TROCANDO DE "Alessandra" para "Psiquiatra" ===
Quantidade de valores únicos antes da unificação:
Valores únicos antes da unificação: 375
Valores que começam com 'psi' antes da unificação: ['Psiquiatra' 'Psicóloga' 'Psicólogo']
=== TROCANDO DE "Psicóloga" para "Psicólogo" ===
Valores que começam com 'psi' depois da unificação: ['Psiquiatra' 'Psicólogo']


# Exportar DataFrame completo para CSV

In [126]:
# # Salvar dataset processado
print("=== SALVANDO DATASET PROCESSADO ===")

# Salvar como CSV processado
output_path = '../data/tasks_processed.csv'
df.to_csv(output_path, index=False, encoding='utf-8')
print(f"✅ Dataset processado salvo em: {output_path}")

# Estatísticas finais
print(f"\n=== ESTATÍSTICAS FINAIS ===")
print(f"Total de registros: {len(df)}")
print(f"Categorias únicas: {df['CALENDAR'].nunique()}")


=== SALVANDO DATASET PROCESSADO ===
✅ Dataset processado salvo em: ../data/tasks_processed.csv

=== ESTATÍSTICAS FINAIS ===
Total de registros: 7232
Categorias únicas: 13
